#### Exploring eBay Car Sales Data
We'll work with dataset of used cars from **eBay Kleinanzeigen**. The original data [set](https://data.world/data-society/used-cars-data)

The aim of this project os to clean the data and analyze the included used car listings.<p>
Over 370000 used cars scraped with Scrapy from Ebay-Kleinanzeigen

    
Those fields are included:
- dateCrawle : when this ad was first crawled, all field-values are taken from this date
- name : "name" of the car
- seller : private or dealer offerType
- price : the price on the ad to sell the car abtest
- vehicleType
- yearOfRegistration : at which year the car was first registered
- gearbox
- powerPS : power of the car in PS
- model
- kilometer : how many kilometers the car has driven
- monthOfRegistration : at which month the car was first registered
- fuelType
- brand
- notRepairedDamage : if the car has a damage which is not repaired yet
- dateCreated : the date for which the ad at ebay was created
- nrOfPictures : number of pictures in the ad postalCode
- lastSeenOnline : when the crawler saw this ad last online

    

The fields lastSeen and dateCrawled could be used to estimate how long a car will be at least online before it is sold.



In [32]:
import pandas as pd
import numpy as np


In [46]:
autos = pd.read_csv("datasets/autos.csv", encoding = "Latin-1") #encoding="UTS-8"
autos.head()
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

### Clean Columns

The dataset contains 20 columns, most of which are string. 
<br>Some columns have null value, but none have more than 15%-20% null values. 
<br>The column names use camelCase instead of Python's preferred shake_case, which means we can't just replace spaces with underscres. 
<br>Make a few change here.

In [47]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [48]:
autos.columns=['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'kilometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'date_created', 'num_pictures', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,date_created,num_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


### Initial Data exploration and cleaning
Text columns where all or almost all value are the same. There can often be dropped as they don't have useful infirmation for analysis.
<br> Examples of numeric data stored as text which can be cleaned and converted.

In [49]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,date_created,num_pictures,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


If u need investigate any columns more `Series.value_counts()` and `Series.head()`.
<br>There are a number of text columns where all(near all) of the values are the same `seller`, `offer_type`. We can mire investigation column names `num_pictures`, `postal_code` looks odd.

In [50]:
autos["num_pictures"].value_counts()
#autos["offer_type"].value_counts()
#autos["seller"].value_counts()

0    371528
Name: num_pictures, dtype: int64

In [51]:
#We can dropped columns
autos=autos.drop(["num_pictures", "offer_type", "seller"], axis=1)

In [54]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,date_created,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,60437,2016-04-06 10:17:21


If we have values column kilometer with estra characters (km, float), we'll clean and convert these.<br>
`autos["meter"] = (autos["meter"]
                             .str.replace("km","")
                             .str.replace(",","")
                             .astype(int)
                             )
autos.rename({"meter": "kilometer"}, axis=1, inplace=True)`
<br><br>

### Exploring Value(Price and Kilometer)
We'll take analyze the columns usinf minimum and maximum value and look for anyvalue that look unrealistically high that we want to remove.
<br>
- `Series.unique().shap` to see how many unique values
- `Series.describe()` to view min/max/median/mean etc
- `Series.value_counts()`, with some variations:
    - chained to `.head()` if there are lots of values.
    - Because `Series.value_counts()` returns a series, we can use `Series.sort_index()` with `ascending= True or False` to view the highest and lowest values with their counts.
- When removing outliers, we can do `df[(df["col"] >= x ) & (df["col"] <= y )]`, but it's more readable to use
`df[df["col"].between(x,y)]`

In [59]:
autos["kilometer"].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: kilometer, dtype: int64

We can see that the values are rounded. There are more high mileage than low mileage vehicles(auto).

In [67]:
print(autos["price"].value_counts().head(10))
print(autos["price"].unique().shape)
print(autos["price"].describe())

0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
Name: price, dtype: int64
(5597,)
count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64


The price seem rounded, there are 5597 unique values in the column. There are 10778 cars listed with 0 price (only 1-2%). the max price is 2 hundred million dillars.

In [69]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
Name: price, dtype: int64

In [70]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
16        2
17        5
18        3
19        3
20       51
Name: price, dtype: int64

There are a number of listings with prices below 20 dollars, including about 10778 at 0 dollars. There are also a small number of listings with very high value, 16 at over 1 billion dollar. <br>
EBAy is an auction site, there could legitimately be opening price is 1 dollar, but we can see that prices increase steadily to the 3,895,000 and them jump up to less realistic number. I gusse we remove anythinf above this number.

In [77]:
auto = autos[autos["price"].between(1,3896000)]
autos["price"].describe()

count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64

### Exploring the data columns
There are 5 columns that should represent data value. `date_crawled`, `registration_year`, `registration_month`, `date_created`, `last_seen`.  Two columns registration are represented as numeric values, others are identified as string value by pandas.
<br>We'll explore each of these columns to learn more about the listings.

In [80]:
autos[['date_crawled','date_created','last_seen']][0:10]

,date_crawled,date_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21
5,2016-04-04 17:36:23,2016-04-04 00:00:00,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2016-04-01 00:00:00,2016-04-05 18:18:39
7,2016-03-21 18:54:38,2016-03-21 00:00:00,2016-03-25 16:47:58
8,2016-04-04 23:42:13,2016-04-04 00:00:00,2016-04-04 23:42:13
9,2016-03-17 10:53:50,2016-03-17 00:00:00,2016-03-31 17:17:06


In [86]:
(autos["date_crawled"]
            .str[:10]
            .value_counts(normalize=True, dropna=False)
            .sort_index()
            )

2016-03-05    0.025551
2016-03-06    0.014462
2016-03-07    0.035690
2016-03-08    0.033454
2016-03-09    0.034210
2016-03-10    0.032563
2016-03-11    0.032727
2016-03-12    0.036194
2016-03-13    0.015735
2016-03-14    0.036275
2016-03-15    0.033451
2016-03-16    0.030148
2016-03-17    0.031661
2016-03-18    0.013127
2016-03-19    0.035292
2016-03-20    0.036353
2016-03-21    0.035739
2016-03-22    0.032469
2016-03-23    0.031971
2016-03-24    0.029912
2016-03-25    0.032934
2016-03-26    0.031965
2016-03-27    0.030275
2016-03-28    0.035112
2016-03-29    0.034170
2016-03-30    0.033529
2016-03-31    0.031885
2016-04-01    0.034116
2016-04-02    0.035079
2016-04-03    0.038735
2016-04-04    0.037612
2016-04-05    0.012823
2016-04-06    0.003163
2016-04-07    0.001620
Name: date_crawled, dtype: float64

In [82]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-04-07    0.001620
2016-04-06    0.003163
2016-04-05    0.012823
2016-03-18    0.013127
2016-03-06    0.014462
2016-03-13    0.015735
2016-03-05    0.025551
2016-03-24    0.029912
2016-03-16    0.030148
2016-03-27    0.030275
2016-03-17    0.031661
2016-03-31    0.031885
2016-03-26    0.031965
2016-03-23    0.031971
2016-03-22    0.032469
2016-03-10    0.032563
2016-03-11    0.032727
2016-03-25    0.032934
2016-03-15    0.033451
2016-03-08    0.033454
2016-03-30    0.033529
2016-04-01    0.034116
2016-03-29    0.034170
2016-03-09    0.034210
2016-04-02    0.035079
2016-03-28    0.035112
2016-03-19    0.035292
2016-03-07    0.035690
2016-03-21    0.035739
2016-03-12    0.036194
2016-03-14    0.036275
2016-03-20    0.036353
2016-04-04    0.037612
2016-04-03    0.038735
Name: date_crawled, dtype: float64

Look like the site was crawled daily over roughly a one month period in March amd April.

In [88]:
(autos["last_seen"]
             .str[:10]
             .value_counts(normalize=True, dropna=False)
             .sort_index()
)

2016-03-05    0.001292
2016-03-06    0.004134
2016-03-07    0.005262
2016-03-08    0.008056
2016-03-09    0.009994
2016-03-10    0.011563
2016-03-11    0.013046
2016-03-12    0.023401
2016-03-13    0.008489
2016-03-14    0.012301
2016-03-15    0.016411
2016-03-16    0.016419
2016-03-17    0.028760
2016-03-18    0.006931
2016-03-19    0.016314
2016-03-20    0.019915
2016-03-21    0.020136
2016-03-22    0.020607
2016-03-23    0.018149
2016-03-24    0.019237
2016-03-25    0.019097
2016-03-26    0.016160
2016-03-27    0.016909
2016-03-28    0.022273
2016-03-29    0.023312
2016-03-30    0.023856
2016-03-31    0.024238
2016-04-01    0.024022
2016-04-02    0.025016
2016-04-03    0.025366
2016-04-04    0.025654
2016-04-05    0.126206
2016-04-06    0.217830
2016-04-07    0.129648
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.
<br>
The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales

In [89]:
print(autos["date_created"].str[:10].unique().shape)
(autos["date_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

(114,)


2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
                ...   
2016-04-03    0.038893
2016-04-04    0.037741
2016-04-05    0.011655
2016-04-06    0.003155
2016-04-07    0.001558
Name: date_created, Length: 114, dtype: float64

There is a large variety of ad created dates. Most fall within 2-4 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [91]:
print(autos["registration_year"].describe())
print(autos["registration_month"].describe())


count    371528.000000
mean       2004.577997
std          92.866598
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64
count    371528.000000
mean          5.734445
std           3.712412
min           0.000000
25%           3.000000
50%           6.000000
75%           9.000000
max          12.000000
Name: registration_month, dtype: float64


The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values. The minimum value is `1000`, long before cars were invented and the maximum is `9999`, many years into the future.
<br>
### Dealing with Incorrect Registration Year Data
Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.
<br>
One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [93]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.03969552765874981

Given that this is less than 4% of our data, we'll remove these rows

In [94]:
# Many ways to select rows in a dataframe that fall within a value range for a column.
# Using `Series.between()` is one way.
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.068813
1999    0.063812
2005    0.062548
2006    0.056702
2001    0.056668
2003    0.055701
2004    0.055345
2002    0.053784
1998    0.050314
2007    0.049535
Name: registration_year, dtype: float64

It appears that most of the vehicles were first registered in the past 20 years.
### Exploring Price by Brand

In [95]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.212391
bmw               0.109695
opel              0.107069
mercedes_benz     0.095955
audi              0.089352
ford              0.068849
renault           0.047559
peugeot           0.029859
fiat              0.025758
seat              0.018631
skoda             0.015407
mazda             0.015348
smart             0.014104
citroen           0.013874
nissan            0.013569
toyota            0.012747
sonstige_autos    0.010648
hyundai           0.009830
mini              0.009210
volvo             0.009129
mitsubishi        0.008266
honda             0.007587
kia               0.006878
alfa_romeo        0.006348
suzuki            0.006318
porsche           0.006149
chevrolet         0.005014
chrysler          0.003941
dacia             0.002450
jeep              0.002186
daihatsu          0.002186
land_rover        0.002136
subaru            0.002125
jaguar            0.001721
trabant           0.001620
saab              0.001452
daewoo            0.001438
r

German manufactures almost 50% of the overall listings. Volkswagen is by far the most popular brend.

There are lots of brends that don't have a significant percentage of listing, we;ll limit our analysis to brands representing more than 5% of total listing.

In [111]:
br_counts=autos["brand"].value_counts(normalize=True)
top_brands = br_counts[br_counts > .04].index
top_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford',
       'renault'],
      dtype='object')

In [105]:
brand_mean_prices = {}

for brand in top_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)
    
brand_mean_prices

{'volkswagen': 13643,
 'bmw': 14798,
 'opel': 3248,
 'mercedes_benz': 17614,
 'audi': 16218,
 'ford': 8702,
 'renault': 2366}

There is a district prise gap:
- Mercedes, Audi, BMW more expensive
- Renault and Opel, Ford are less expensive
- Volkswagen is in between



### Exploring Mileage

In [116]:
#combine the data from both series object into a single dataframe(series constructor)
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns= ["mean_price"])

,mean_price
volkswagen,13643
bmw,14798
opel,3248
mercedes_benz,17614
audi,16218
ford,8702
renault,2366


In [117]:
brand_mean_mileage = {}

for brand in top_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["kilometer"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [118]:
brand_info = pd.DataFrame(mean_mileage, columns = ["mean_mileage"])
brand_info

,mean_mileage
bmw,132657
mercedes_benz,130580
audi,129491
opel,128755
volkswagen,128337
renault,127875
ford,123618


In [119]:
brand_info["mean_price"] = mean_prices
brand_info

,mean_mileage,mean_price
bmw,132657,14798
mercedes_benz,130580,17614
audi,129491,16218
opel,128755,3248
volkswagen,128337,13643
renault,127875,2366
ford,123618,8702


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage